Digital Steganography is method of hiding secret messages inside pictures.  Let's start with a random image.  Run the next code sample to display the image.

In [ ]:
from PIL import Image
im = Image.open("puppy.jpg")
display(im)

We are going to hide a secret message in this image, by replacing the least significant bit of the red channel of each pixel with one bit from our message.  This image is 800 pixels by 600 pixels, so you could store a message up to 480,000 characters long with this method, about 6x the length of _Macbeth_.  Run the next code sample to embed a message in the photo.

In [ ]:
from PIL import Image
im = Image.open("puppy.jpg")
SecretMessage="This is a secret message.  Good job finding it."

my_str_as_bytes = str.encode(SecretMessage)

widthCounter = 32;
heightCounter = 0;
pix=im.load()
#first 8 contain the length
length = len(my_str_as_bytes)
#assumes the picture is at least 32 pixels wide
for i in range(0,32):
    oldValue = pix[0,i][0]
    newValue = oldValue
    if( (length & (1 << i)) != 0 ):#bit is 1
          newValue = newValue | 1
    else:#bit is 0
            newValue = newValue & ~1
    pix[0,i] = (newValue,pix[0,i][1],pix[0,i][2])
    
for i in range(0,len(my_str_as_bytes)*8):#8 bits in a byte
    oldValue = pix[heightCounter,widthCounter][0]
    newValue = oldValue
    if( (my_str_as_bytes[i//8] & (1 << (i%8))) != 0):
        newValue = newValue | 1
    else:#bit is 0
            newValue = newValue & ~(1)
    pix[heightCounter,widthCounter] = (newValue,pix[heightCounter,widthCounter][1],pix[heightCounter,widthCounter][2])
    widthCounter += 1
    if(widthCounter == im.width):
             widthCounter=0
             heightCounter += 1
                
im.save('out.bmp')
im2 = Image.open("out.bmp")
display(im2)



This new image has your secret message hidden in it.  I bet you can't tell by looking at it.  You can change the SecretMessage in the previous code sample to hide a new message.  Run the next code sample to extract your hidden message.

In [ ]:
from PIL import Image
im = Image.open("out.bmp")

pix = im.load()

length = 0
for i in range(0,32):
    length = length | ((pix[0,i][0] & 1) << i)
    
widthCounter = 32
heightCounter = 0
my_str_as_bytes = bytearray()
for i in range(0,length):#8 bits in a byte
    character = 0
    for j in range(0,8):
        character = character | ((pix[heightCounter,widthCounter][0] & 1) << j)
        widthCounter = widthCounter + 1
        if(widthCounter == im.width):
            heightCounter = heightCounter + 1
    my_str_as_bytes.append(character)
        
my_decoded_str = my_str_as_bytes.decode()
print(my_decoded_str)

If you have a friend that also has this Notebook, send them your picture and see if they can decode your message!